## Reflection Pattern for LLM
This notebook demonstrates the Reflection pattern for LLMs as defined by Andrew Ng of DeepLearning.ai.  The goal is to show how to prompt an LLM, have another prompt "reflect" on the answer, and incorporate the revisions from the reflection.

This examples demostrates how to generate python code for a Merge Sort (Classic Sorting Algo).

Initialize the packages needed for this notebook. Including:
* streamlit functions for display
* pandas
* cortext

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from IPython.display import display_markdown

from snowflake.cortex import Complete
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
def build_prompt_structure (prompt: str, role: str, tag: str = "") -> dict:
    """
    Builds a structured prompt that includes the role and content.

    Args:
        prompt (str): The actual content of the prompt.
        role (str): The role of the speaker (e.g., user, assistant).

    Returns:
        dict: A dictionary representing the structured prompt.
    """
    if tag:
        prompt = f"<{tag}>{prompt}</{tag}>"
    return {"role": role, "content": prompt}

In [ ]:
build_prompt_structure(prompt = "write  merge sort algorithm in python", role = "user")

In [ ]:
query = st.text_area("Enter your prompt:","Generate a Python implementation of the Merge Sort algorithm", 
                     height=150)


In [ ]:
print (query)

Build system prompt that generates code.
Also create an array to store the history of the code generatation prompts.

In [ ]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the users request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]
generation_chat_history.append(
    {
        "role": "user",
        "content": query
    }
)

st.markdown (generation_chat_history)

### Generate the code
* Use llama to "complete" the prompt.
* capture the output in the "mergesort_code" variable

In [ ]:
mergesort_code = Complete ('llama3.1-405b', generation_chat_history)

Use the streamlit function "markdown" to show the formatted code.

In [ ]:
st.markdown(mergesort_code)

In [ ]:
generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [ ]:
print (generation_chat_history)

In [ ]:
reflection_chat_history = [
    {
        "role":"system",
        "content" : "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user code. If unit test are not, present suggest to create unit tests"
    }
]

In [ ]:
reflection_chat_history.append (
    {
        "role":"user",
        "content": mergesort_code
    }
)
print (reflection_chat_history)

In [ ]:
critique = Complete ('llama3.1-405b', reflection_chat_history)

In [ ]:
st.markdown (critique)


## Incorporate the critique
The next step is to update the merge_code with the critique from our computer scientist.
Then display the final implementaiton.

In [ ]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

In [ ]:
essay = Complete ('llama3.1-405b', generation_chat_history)

In [ ]:
st.markdown(essay)